In [1]:
import pandas as pd
import numpy as np
import pickle

dataframes = pd.read_pickle('recom_data/user_category_dummy.pkl')

df_counts = dataframes['view_counts']
df_users = dataframes['users']

with open('recom_data/category_dict.pkl', 'rb') as handle:
    category_dict = pickle.load(handle)

item_similarity = pd.read_pickle('recom_data/item_similarity.pkl')

In [2]:
view_matrix = df_counts.pivot(index='user', columns='category', values='count')

In [97]:
def transform_gender(gender):
    return 'M' if (gender == '남자') else 'F'

def transform_age(age):
    return (age // 10) * 10

def recom_simple1(user, n_items=20):
    user_gender = transform_gender(user['gender'])
    user_age = transform_age(user['age'])

    mask_gender = df_users['gender'] == user_gender
    mask_age = df_users['age'] == user_age

    user_group = df_users[mask_gender & mask_age]
    
    mask_group = new_df['user'].isin(user_group['user'])
    view_df = new_df[mask_group]
    
    grouped_view_df = (view_df.groupby(['category'])['count']
                       .agg(['sum'])
                       .sort_values(by=['sum'], ascending=False)
                       .apply(lambda x: x / len(view_df['user'].unique()))
                       .reset_index())
    
    return grouped_view_df

def recom_simple2(user, n_items=20):    
    grouped_view_df = recom_simple1(user, n_items=n_items)
    
    for tester_view in tester_view_data:
        if tester_view[1] in grouped_view_df['category'].values:
            grouped_view_df.loc[grouped_view_df.category==tester_view[1], 'sum'] += tester_view[2]
        else:
            
            grouped_view_df = grouped_view_df.append({'category': tester_view[1], 'sum': tester_view[2]}, ignore_index=True)
    
    grouped_view_df = grouped_view_df.sort_values(by=['sum'], ascending=False)
    return grouped_view_df

tester1 = {'user_id': 1, 'gender': '남자', 'age': 27}

result1 = recom_simple1(tester1)
result1['category'] = result1['category'].apply(lambda x: category_dict[x])
print(result1)

tester2 = {'user_id': 2, 'gender': '남자', 'age': 22}
tester_view_data = [[2, 11, 1], [2, 12, 2], [2, 1, 920]]
tester_df = pd.DataFrame(tester_view_data, columns=['user', 'category', 'count'])
new_df = df_counts.append(tester_df)

result2 = recom_simple2(tester2)
result2['category'] = result2['category'].apply(lambda x: category_dict[x])
print(result2)

       category     sum
0        스마트 밴드  1.7250
1        스마트 워치  1.4625
2        남성용 가방  1.2000
3       남성 캐주얼화  1.1750
4         남성 상의  1.1500
..          ...     ...
144       헤어 왁스  0.2250
145       퍼즐/블록  0.2125
146       골프 클럽  0.2125
147     인라인스케이트  0.1750
148  기타 스케이트 용품  0.1500

[149 rows x 2 columns]
       category       sum
149       여성 상의  920.0000
5         남성 하의    3.1125
4         남성 상의    2.1500
0        스마트 밴드    1.7250
1        스마트 워치    1.4625
..          ...       ...
144       헤어 왁스    0.2250
145       퍼즐/블록    0.2125
146       골프 클럽    0.2125
147     인라인스케이트    0.1750
148  기타 스케이트 용품    0.1500

[150 rows x 2 columns]


In [ ]:
print(tester['gender'], tester['age'])
print(simple_recommend(tester))
testers = [{'gender': '남자', 'age': 10}, {'gender': '남자', 'age': 20}, {'gender': '남자', 'age': 30}, {'gender': '남자', 'age': 40}, {'gender': '남자', 'age': 50}, {'gender': '남자', 'age': 60}, {'gender': '남자', 'age': 70},
           {'gender': '여자', 'age': 10}, {'gender': '여자', 'age': 20}, {'gender': '여자', 'age': 30}, {'gender': '여자', 'age': 40}, {'gender': '여자', 'age': 50}, {'gender': '여자', 'age': 60}, {'gender': '여자', 'age': 70}]

for test in testers:
    print(test['gender'], test['age'])
    print(simple_recommend(test))

In [4]:
from sklearn.model_selection import train_test_split
x = df_counts.copy()
y = df_counts['user']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [5]:
# RMSE 계산해주는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true)- np.array(y_pred)) ** 2))

# 모델별 RMSE 계산
def score(model):
    id_pairs = zip(x_test['user'], x_test['category'])
    y_pred = np.array([model(user, category) for (user, category) in id_pairs])
    y_true = np.array(x_test['count'])
    return RMSE(y_true, y_pred)

train_view_matrix = x_train.pivot(index='user', columns='category', values='count')
view_matrix_t = np.transpose(train_view_matrix)

In [6]:
def simple_all_model(user, category):
    try:
        pred_viewing = train_mean[category]
    except:
        pred_viewing = 0
    return pred_viewing

user_count = len(x_train['user'].unique())
train_mean = x_train.groupby(['category'])['count'].sum().apply(lambda x: x/user_count)

# print(score(simple_all_model))

In [7]:
def IBCF_model(user, category):
    if category in item_similarity:
        sim_scores = item_similarity[category]
        user_viewing = view_matrix_t[user]
        non_viewing_idx = user_viewing[user_viewing.isnull()].index
        user_viewing = user_viewing.dropna()
        sim_scores = sim_scores.drop(non_viewing_idx)
        pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    else:
        pred_viewing = 0
    return pred_viewing

# print(score(IBCF_model))

In [14]:
test_user = {
    'gender': '여성',
    'age': 26
}

test_user_view_data = [
    [1, 1, 4],
    [1, 2, 2],
    [1, 3, 2],
    [1, 4, 1],
    [1, 31, 1],
    [1, 32, 1],
    [1, 33, 3]
]

temp_user_df = pd.DataFrame(test_user_view_data, columns=['user', 'category', 'count'])
new_df = df_counts.append(temp_user_df)

view_matrix = new_df.pivot(index='user', columns='category', values='count')

In [252]:
def recom_category(user_id, n_items=20):
    for category in view_matrix:
        user_category.loc[category] = IBCF_model(user_id, category)
    category_sort = user_category.sort_values(ascending=False)[:n_items]
    return category_sort

def IBCF_model(user, category):
    if category in item_similarity:
        sim_scores = item_similarity[category]
        user_viewing = user_category.T
        non_viewing_idx = user_viewing[user_viewing.isnull()].index
        user_viewing = user_viewing.dropna()
        sim_scores = sim_scores.drop(non_viewing_idx)
        pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    else:
        pred_viewing = 0
    return pred_viewing

user_category = view_matrix.loc[90021].copy()

pred_user_category = recom_category(user_id=90021, n_items=10)
recommend_category = pd.DataFrame(pred_user_category).index.map(lambda x: category_dict[x])
print(list(recommend_category))

['남성 상의', '남성용 모자', '카시트', '이유식', '아기 간식', '유아 세제', '구강청결 용품', '소독/살균 용품', '스킨/바디 용품', '기타 유아 외출 용품']


In [123]:
def recom_simple3(user, n_items=423):    
    grouped_view_df = recom_simple1(user, n_items=423)
    
    for tester_view in test_user_view_data:
        if tester_view[1] in grouped_view_df['category'].values:
            grouped_view_df.loc[grouped_view_df.category==tester_view[1], 'sum'] += tester_view[2]
        else:
            
            grouped_view_df = grouped_view_df.append({'category': tester_view[1], 'sum': tester_view[2]}, ignore_index=True)
    
    grouped_view_df = grouped_view_df.sort_values(by=['sum'], ascending=False)
    return grouped_view_df


pred1_user_category = recom_simple3(test_user, n_items=423)['sum']
true1_user_category = pd.DataFrame(test_user_view_data, columns=['user', 'category', 'sum']).drop(['user'], axis=1).set_index('category')['sum']
merged = pd.merge(pred1_user_category, true1_user_category, left_index=True, right_index=True, how='outer', suffixes=['_pred', '_true']).fillna(0)
print(RMSE(merged['sum_pred'], merged['sum_true']))
# 기존 그룹화 추천의 경우 데이터가 없는 경우는 Error가 낮게 나올 수밖에 없음
# 모델과 에러가 큰 차이가 안 날 경우 전환하는 걸로 얘기해야 될 듯....

true2_user_category = view_matrix.loc[1].fillna(0)
pred2_user_category = recom_movie(user_id=1, n_items=423)
print(RMSE(pred2_user_category, true2_user_category))

0.8213529418466912
1.8483182832550367


In [211]:
class MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        """
            K: 잠재요인(latent factor)의 수
            alpha: 학습률
            beta: 정규화 계수
            iterations: SGD의 계산을 할 때의 반복 횟수
            verbose: SGD의 중간 학습과정 출력 여부
        """
        self.R = np.array(ratings)
        
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings):
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)
        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)
        
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose
        
    def rmse(self):
        """
            현재의 P행렬과 Q행렬을 가지고 Root Mean Squared Error(RMSE)를 계산
        """
        xs, ys = self.R.nonzero()
        # R에서 평점이 있는 (0이 아닌) 요소의 인덱스를 저장
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            # 평점이 존재하는 요소 (사용자 x, 아이템 y) 각각에 대해 아래 코드를 실행
            prediction = self.get_prediction(x, y)
            # 사용자 x, 아이템 y에 대한 평점 예측치를 계산
            self.predictions.append(prediction)
            # 예측값을 리스트에 추가
            self.errors.append(self.R[x, y] - prediction)
            # 실제값과 예측값의 차이를 계산해서 오차값 리스트에 추가
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        # errors를 사용해서 RMSE를 계산
        return np.sqrt(np.mean(self.errors**2))
    
    def train(self):
        """
            정해진 횟수만큼 반복하며 P, Q, bu, bd 값을 업데이트하는 함수
        """
        # Initializing user-feature and movie-feature matrix
        # P, Q 행렬을 임의의 값으로 채움
        # 평균이 0, 표준편차가 1/K인 정규분포를 갖는 난수로 초기화
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initializing the bias terms
        # 사용자 평가 경향(bu)를 0으로 초기화 / 크기는 사용자 수 (num_users)
        # 아이템 평가 경향(bd)를 0으로 초기화 / 크기는 아이템 수 (num_items)
        # 전체 평균 b를 구해서 저장
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])
        
        # List of training samples
        # 평점 행렬에서 값이 존재하는 요소의 인덱스들을 가져옴
        # SGD를 적용할 대상 (평점이 있는 요소의 인덱스와 평점)을 리스트로 저장
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]
        
        # Stochastic gardient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            # samples를 임의로 섞음 => 어디서 시작하는지에 따라 수렴의 속도가 달라짐
            # 매 반복마다 다양한 시작점에서 시작하기!
            self.sgd()
            # sgd를 실행 / P, Q, bu, bd가 업데이트 됨
            rmse = self.rmse()
            training_process.append((i+1, rmse))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("반복: {0}; Train RMSE = {1:0.4f}".format(i+1, rmse))
        return training_process
    
    def get_prediction (self, i, j):
        """
            Rating prediction for user i and item j
            사용자 i의 아이템 j에 대한 평점을 예측하는 함수
        """
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
        
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])
    
    def full_prediction(self):
        return self.b + self.b_u[:, np.newaxis] + self.b_d[np.newaxis, :] + self.P.dot(self.Q.T)

    def sgd(self):
        """
            Stochastic gardient descent to get optimized P and Q matrix
        """
        for i, j, r in self.samples:
            # samples에 있는 각 사용자-아이템-평점 세트에 대해서 SGD를 적용
            prediction = self.get_prediction(i, j)
            # 사용자 i, 아이템 j에 대한 평점 예측치를 구함
            e = (r - prediction)
            # 실제 평점 r과 비교해서 오차를 구함
            
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])
            # 사용자 평가 경향, 아이템 평가 경향 업데이트
            
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j, :])

In [213]:
view_matrix = df_counts.pivot(index='user', columns='category', values='count')
R_temp = view_matrix.fillna(0)
mf = MF(R_temp, K=40, alpha=0.001, beta=0.02, iterations=100, verbose=True)
mf.train()

반복: 10; Train RMSE = 0.8642
반복: 20; Train RMSE = 0.8576
반복: 30; Train RMSE = 0.8543
반복: 40; Train RMSE = 0.8522
반복: 50; Train RMSE = 0.8505
반복: 60; Train RMSE = 0.8490
반복: 70; Train RMSE = 0.8474
반복: 80; Train RMSE = 0.8455
반복: 90; Train RMSE = 0.8430
반복: 100; Train RMSE = 0.8396


[(1, 0.8811825066506646),
 (2, 0.8779063295867424),
 (3, 0.8751811320149158),
 (4, 0.872872553762934),
 (5, 0.8708989780637395),
 (6, 0.8691950106450145),
 (7, 0.8677029345532578),
 (8, 0.8663864469275141),
 (9, 0.8652164805305447),
 (10, 0.8641691845709066),
 (11, 0.8632260262852854),
 (12, 0.8623715559121833),
 (13, 0.8615932718464199),
 (14, 0.8608822301269149),
 (15, 0.8602292402796832),
 (16, 0.8596266333272398),
 (17, 0.8590688543444183),
 (18, 0.85855114802825),
 (19, 0.8580684600594233),
 (20, 0.8576171151694795),
 (21, 0.8571949182805554),
 (22, 0.856798443644169),
 (23, 0.8564252629249071),
 (24, 0.8560729239362352),
 (25, 0.8557394797538432),
 (26, 0.8554236815572213),
 (27, 0.8551230455032836),
 (28, 0.85483687907574),
 (29, 0.8545640665083042),
 (30, 0.854303167139095),
 (31, 0.8540532168454588),
 (32, 0.8538137838008204),
 (33, 0.8535837627138791),
 (34, 0.8533621677569256),
 (35, 0.8531487316843885),
 (36, 0.8529420860546526),
 (37, 0.8527428462870397),
 (38, 0.852548527

In [214]:
print(mf.full_prediction())

[[2.18752574 2.31195994 2.71366166 ... 2.35788963 2.07577267 1.88747403]
 [2.09647954 2.17035065 2.597129   ... 2.22947302 1.97117349 1.76833088]
 [2.07825045 2.18545086 2.61048895 ... 2.24506848 1.98288988 1.782311  ]
 ...
 [2.14947492 2.26808975 2.67486957 ... 2.30187034 2.0376123  1.85150046]
 [2.15767959 2.27710264 2.67811186 ... 2.29416096 2.04118858 1.85299752]
 [2.15300576 2.28478669 2.68643063 ... 2.28234866 2.04438857 1.85270805]]


In [249]:
tester = {
    'user_id': 90021,
    'gender': '남성',
    'age': 22
}

temp_data = dict()
for category in R_temp:
    temp_data[category] = mf.get_one_prediction(tester['user_id'], category)

recomm_df = pd.DataFrame.from_dict([temp_data]).T.reset_index().rename(columns={'index': 'category', 0: 'pred'})
print(recomm_df)
temp_recomm_df = recomm_df.sort_values(by='pred', ascending=True)
temp_recomm_df['category'] = temp_recomm_df['category'].map(lambda x: category_dict[x])
print(temp_recomm_df)

     category      pred
0           1  1.930796
1           2  2.043944
2           3  2.426131
3           4  2.086591
4          11  1.992366
..        ...       ...
418      1092  1.963692
419      1093  1.796570
420      1094  2.043013
421      1101  1.803813
422      1102  1.615544

[423 rows x 2 columns]
     category      pred
259       샤워기  1.215086
235    기타 수집품  1.247878
237    관상어 수조  1.252270
286       옷걸이  1.254805
260     샤워 용품  1.262105
..        ...       ...
315    유아동 의류  2.178017
324   유아발육 용품  2.206583
302  소독/살균 용품  2.216371
326     교재/서적  2.244282
2       여성 잠옷  2.426131

[423 rows x 2 columns]


In [253]:
def recom_category(user_id, n_items=20):
    for category in view_matrix:
        user_category.loc[category] = IBCF_model(user_id, category)
    return user_category.reset_index().rename(columns={user_id: 'pred'})

def IBCF_model(user, category):
    if category in item_similarity:
        sim_scores = item_similarity[category]
        user_viewing = user_category.T
        non_viewing_idx = user_viewing[user_viewing.isnull()].index
        user_viewing = user_viewing.dropna()
        sim_scores = sim_scores.drop(non_viewing_idx)
        pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    else:
        pred_viewing = 0
    return pred_viewing

user_category = view_matrix.loc[90021].copy()

pred_user_category = recom_category(user_id=90021, n_items=423)
# recommend_category = pd.DataFrame(pred_user_category).index.map(lambda x: category_dict[x])
print(pred_user_category)

temp_temp_temp = pred_user_category.sort_values(by='pred')
temp_temp_temp['category'] = temp_temp_temp['category'].map(lambda x: category_dict[x])
print(temp_temp_temp)

     category      pred
0           1  1.136820
1           2  1.176781
2           3  1.277355
3           4  1.281721
4          11  1.462683
..        ...       ...
418      1092  1.193329
419      1093  1.194008
420      1094  1.189185
421      1101  1.204951
422      1102  1.243335

[423 rows x 2 columns]
     category      pred
352       배구공  0.825195
354     배구 네트  0.852272
351  기타 농구 용품  0.866265
353       배구화  0.874112
355  기타 배구 용품  0.881798
..        ...       ...
298     아기 간식  1.314586
297       이유식  1.316941
309       카시트  1.318757
31     남성용 모자  1.319112
4       남성 상의  1.462683

[423 rows x 2 columns]


In [256]:
merged = pd.merge(pred_user_category, recomm_df, left_on='category', right_on='category', how='outer', suffixes=['_IBCF', '_MF'])
merged['pred'] = merged['pred_IBCF'] * 0.9 + merged['pred_MF'] * 0.1
merged['category'] = merged['category'].map(lambda x: category_dict[x])
print(merged.sort_values(by='pred'))

     category  pred_IBCF   pred_MF      pred
352       배구공   0.825195  1.783882  0.921063
354     배구 네트   0.852272  1.826940  0.949738
351  기타 농구 용품   0.866265  1.705580  0.950197
355  기타 배구 용품   0.881798  1.661918  0.959810
353       배구화   0.874112  1.807160  0.967416
..        ...        ...       ...       ...
308       유모차   1.311220  2.162858  1.396384
324   유아발육 용품   1.306591  2.206583  1.396590
326     교재/서적   1.306187  2.244282  1.399996
302  소독/살균 용품   1.313686  2.216371  1.403954
4       남성 상의   1.462683  1.992366  1.515651

[423 rows x 4 columns]
